In [1]:
import requests
from bs4 import BeautifulSoup

import selenium
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

from datetime import datetime as dt
import pandas as pd
import time

In [2]:
def number_of_pages(url):
    # finds the number of the last page of the listings
    
    driver = webdriver.Chrome()
    driver.get(url)
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable(
        (By.XPATH, '//*[@id="search-toolbars-1"]/div[2]/button[4]/span[1]')))
    time.sleep(2)
    driver.find_element(By.CLASS_NAME, "cl-last-page").click()
    time.sleep(1)
    last_page = last_page = driver.current_url.split('gallery~')[1].split('~')[0]
    print(f'The number of pages is {last_page}')
    driver.quit()
    return int(last_page)

In [3]:
def get_links(url, pages_number):
    # gets the urls for all listings
    
    driver = webdriver.Chrome()
    driver.get(url)

    links = [] # creating list to store the links
    for i in range(pages_number + 1):
        WebDriverWait(driver, 10).until(EC.presence_of_element_located(
            (By.XPATH, '//*[@id="search-results-page-1"]/ol/li[40]')))
        WebDriverWait(driver, 10).until(EC.presence_of_element_located(
            (By.CLASS_NAME, 'cl-next-page')))
        
        soup = BeautifulSoup(driver.page_source, 'html')
        page_listings = soup.find_all(attrs={'class':'cl-search-result cl-search-view-mode-gallery'})
        links_temp = [l.find('a').get('href') for l in page_listings]
        links.append(links_temp)
        
        WebDriverWait(driver, 10).until(EC.presence_of_element_located(
            (By.CLASS_NAME, 'cl-next-page')))
        driver.find_element(By.CLASS_NAME, 'cl-next-page').click()
        
    driver.quit()
    print("All listings' links are successfully extracted")
    
    # flattening the list and removing duplicates
    links_flat = list(set([i for temp_list in links for i in temp_list]))
    
    return links_flat

In [4]:
def scrap_listings(links):
    print('This operation may take a few hours')
    listings_raw = []
    bad_connection_urls = [] # stores the links that are not scraped
    count = 1
    for l in links:
        # we use try/except block to avoid a crash if the internet is unstable,
        # or if the response code is different from 200 or other reasons
        try: 
            response = requests.get(l)
            print(l)
            print(f'The response code is {response.status_code}')
            soup = BeautifulSoup(response.content, "html.parser")

            title = soup.find(attrs={'class': 'postingtitletext'})
            title_list = []
            for t in title.find_all('span'):
                title_list.append(t.text.strip())

            content = soup.find(attrs={'id': 'postingbody'})
            content_list = []
            for c in content.contents:
                if c.text.strip() != '':
                    content_list.append(c.text.strip())

            attributes_list = []
            attributes = soup.find_all(attrs={'class': 'attrgroup'})
            for a in attributes:
                if a.text.strip() != '':
                    attributes_list.append(a.text.strip())

            location_list = []
            location = soup.find(attrs={'id': 'map'})
            location_list.append(location.attrs['data-latitude'])
            location_list.append(location.attrs['data-longitude'])

            listings_raw.append([l ,title_list, content_list, attributes_list, location_list])

            print(f'{count} out of {len(links)}')
        except:
            print('bad connection')
            bad_connection_urls.append(l)
        count += 1
        time.sleep(1) # giving a delay of а secоnd not to get blocked by the server
        
    return listings_raw, bad_connection_urls

In [5]:
def get_attributes(scrap_list):
    # extracts attributes to a list
    listings_raw = []
    for i in scrap_list:
        # placing condition to eliminate listings without enough information
        if len(i[-2]) > 0 and 'BR / ' in i[3][0]:
            
        # appending the url, price, area, location, number of bedrooms and bathrooms
            listings_raw.append([i[0], i[1][0], i[1][1], i[1][-1],
                                 i[-1][0], i[-1][1], i[3][0], i[2]])  
    
    listings = [['url', 'price', 'area', 'location', 
                 'latitude', 'longitude', 'bedrooms', 'bathrooms', 'description']]
    for i in listings_raw:   
        temp_list = []
        temp_list.append(i[0])
        temp_list.append(i[1].strip('$').replace(',', ''))
        temp_list.append(i[2][-10:-3])
        temp_list.append(i[3].strip('(').strip(')'))
        temp_list.append(i[4])
        temp_list.append(i[5])
        temp_list.append(i[6].split(' / ')[0].strip('BR'))
        temp_list.append(i[6].split(' / ')[1].split('\n')[0].strip('Ba'))
        temp_list.append(i[7])
        listings.append(temp_list)  
        
    return listings

In [6]:
def get_features(scrap_list, show_unique_features=False):
    # getting the listed features on a separate list 
    # and removing empty values and other irrelevant features
    features_temp = [i[-2][-1].split('\n') for i in scrap_list 
                     if len(i[-2]) > 0 and 'BR / ' in i[3][0]]
    
    features_raw = []
    for f in features_temp:
        temp_list = []
        for ff in f:
            if ff != '' and ff.startswith('application') == False and \
            ff.startswith('listed by') == False and ff.startswith('broker') == False \
            and ff.startswith('license') == False:
                temp_list.append(ff)
        features_raw.append(temp_list)
        
    # extracting the unique features
    features_unq = set([i for temp_list in features_raw for i in temp_list])
    if show_unique_features == True:
        print(f'The unique features are {features_unq}')
    
    # assigning column names for the features
    features = [['property_type', 'parking', 'laundry', 'rent_period', 'EV_charging', 
                 'air_conditioning','furnished', 'smoking', 'wheelchair_accessible',
                 'cats_allowed', 'dogs_allowed', 'room_type', 'bath_type']]
    
    # creating the category of the features
    for i in features_raw:
        temp_list = []
        # getting the type of property    
        if 'apartment' in i:
            temp_list.append('apartment')
        elif 'condo' in i:
            temp_list.append('condo')
        elif 'cottage/cabin' in i:
            temp_list.append('cottage/cabin')
        elif 'duplex' in i:
            temp_list.append('duplex')
        elif 'flat' in i:
            temp_list.append('flat')
        elif 'house' in i:
            temp_list.append('house')
        elif 'in-law' in i:
            temp_list.append('in-law')
        elif 'land' in i:
            temp_list.append('land')
        elif 'loft' in i:
            temp_list.append('loft')
        elif 'manufactured' in i:
            temp_list.append('manufactured')
        elif 'townhouse' in i:
            temp_list.append('townhouse')
        elif 'assisted living' in i:
            temp_list.append('assisted living')
        else:
            temp_list.append('n/a')  
        # getting the type of parking
        if 'attached garage' in i:
            temp_list.append('attached garage')
        elif 'carport' in i:
            temp_list.append('carport')
        elif 'detached garage' in i:
            temp_list.append('detached garage')
        elif 'no parking' in i:
            temp_list.append('no parking')
        elif 'off-street parking' in i:
            temp_list.append('off-street parking')
        elif 'street parking' in i:
            temp_list.append('street parking')
        elif 'valet parking' in i:
            temp_list.append('valet parking')
        else:
            temp_list.append('n/a')  
        # getting the type of laundry
        if 'laundry in bldg' in i:
            temp_list.append('laundry in bldg')
        elif 'laundry on site' in i:
            temp_list.append('laundry on site')
        elif 'no laundry on site' in i:
            temp_list.append('no laundry on site')
        elif 'w/d hookups' in i:
            temp_list.append('w/d hookups')
        elif 'w/d in unit' in i:
            temp_list.append('w/d in unit')
        else:
            temp_list.append('n/a')
        # getting the rent period
        if 'rent period: daily' in i:
            temp_list.append('daily')
        elif 'rent period: weekly' in i:
            temp_list.append('weekly')
        elif 'rent period: monthly' in i:
            temp_list.append('monthly')
        else:
            temp_list.append('n/a')
        # if there is EV charging, we assign 1
        if 'EV charging' in i:
            temp_list.append(1)
        else:
            temp_list.append(0)
        # if there is air conditioning, we assign 1
        if 'air conditioning' in i:
            temp_list.append(1)
        else:
            temp_list.append(0)
        # if the property is furnished, we assign 1
        if 'furnished' in i:
            temp_list.append(1)
        else:
            temp_list.append(0)
        # if smoking is allowed, we assign 1
        if 'no smoking' in i:
            temp_list.append(0)
        else:
            temp_list.append(1)
        # if the property is wheelchair accessible, we assign 1
        if 'wheelchair accessible' in i:
            temp_list.append(1)
        else:
            temp_list.append(0)
        # if cats are allowed, we assign 1
        if 'cats are OK - purrr' in i:
            temp_list.append(1)
        else:
            temp_list.append(0)
        # if dogs are allowed, we assign 1
        if 'dogs are OK - wooof' in i:
            temp_list.append(1)
        else:
            temp_list.append(0)
        # this option is only available for shared properties given by rooms
        if 'private room' in i:
            temp_list.append('private room')
        elif 'room not private' in i:
            temp_list.append('room not private')
        else:
            temp_list.append('n/a')
            
        if 'private bath' in i:
            temp_list.append('private bath')
        elif 'not private bath' in i:
            temp_list.append('not private bath')
        else:
            temp_list.append('n/a')
            
        features.append(temp_list)
    return features

In [7]:
def df_to_file(file_name, attributes, features):
    # creates data frame and saves it to csv file
    data_list = []
    for i in range(len(attributes)):
        data_list.append(attributes[i] + features[i])
    df = pd.DataFrame(data_list[1:], columns=data_list[0])
    
    # removing same listings posted more than once listings
    duplicates = df.shape[0]
    df = df.drop_duplicates(subset=['price', 'location', 'bedrooms', 'bathrooms', 'property_type', 
                     'parking', 'laundry', 'rent_period', 'EV_charging', 'air_conditioning',
                     'furnished', 'smoking', 'wheelchair_accessible', 'cats_allowed',
                     'dogs_allowed'], keep='first')
    
    print(f'----- Removing {duplicates - df.shape[0]} duplicates from the dataframe -----')
    
    df = df.reset_index(drop=True)
    
    df.to_csv(file_name)

In [9]:
def app(url, file_name, rerun_bad_connection=True):
    # runs the program
    # The url is copied from craigslist.com --> housing --> apts / housing
    # This scraper can use any location and any type of housing
    # file_name is a name of csv file to be saved on 
            # (it adds the today's date and the file extension automatically)
    # rerun_bad_connection argument reruns urls that weren't scraped the first time
    
    pages_number = number_of_pages(url)
    # NOTE: sometimes it shows the number of pages is 2, so please rerun the app again in such a case
    links = get_links(url, pages_number)
    scrap, bad_connection_url = scrap_listings(links)
    scrap_bad_connection = [] # stores listings which had bad connection
    if (rerun_bad_connection == True) & (len(bad_connection_url) > 0):
        print(f'Rerung {len(bad_connection_url)} urls')
        scrap_bad_connection = scrap_listings(bad_connection_url)[0]
        scrap = scrap + scrap_bad_connection
        
    attributes = get_attributes(scrap)
    features = get_features(scrap)
    # Saving the df to csv and adding the today's date to the file_name
    df_to_file(f'{file_name}_{dt.now().strftime("%Y-%m-%d")}.csv', attributes, features)
    print("All done")


app('https://sfbay.craigslist.org/search/apa#search=1~gallery~0~0', 'apt_house_sfbay')

The number of pages is 83
All listings' links are successfully extracted
This operation may take a few hours
https://sfbay.craigslist.org/sfc/apa/d/san-francisco-modern-apartments-in-the/7600709800.html
The response code is 200
1 out of 9280
https://sfbay.craigslist.org/pen/apa/d/menlo-park-beautiful-2b-1b-house-for/7600412669.html
The response code is 200
2 out of 9280
https://sfbay.craigslist.org/pen/apa/d/palo-alto-charming-spacious-bd-apt-in/7600349518.html
The response code is 200
3 out of 9280
https://sfbay.craigslist.org/nby/apa/d/santa-rosa-large-bedroom-bathroom/7594758521.html
The response code is 200
4 out of 9280
https://sfbay.craigslist.org/sfc/apa/d/san-francisco-beautiful-3bd3ba/7593318187.html
The response code is 200
5 out of 9280
https://sfbay.craigslist.org/eby/apa/d/fairfield-package-concierge-assigned/7599968967.html
The response code is 200
6 out of 9280
https://sfbay.craigslist.org/sfc/apa/d/san-francisco-1950large-studio-20th-ave/7598802697.html
The response cod

https://sfbay.craigslist.org/sfc/apa/d/san-francisco-spacious-studio-apartment/7597986039.html
The response code is 200
63 out of 9280
https://sfbay.craigslist.org/eby/apa/d/oakland-courtesy-patrol-bocce-ball/7600682180.html
The response code is 200
64 out of 9280
https://sfbay.craigslist.org/eby/apa/d/walnut-creek-move-in-special-bedroom/7600655236.html
The response code is 200
65 out of 9280
https://sfbay.craigslist.org/eby/apa/d/fairfield-great-neighborhood-plush/7600722298.html
The response code is 200
66 out of 9280
https://sfbay.craigslist.org/pen/apa/d/san-mateo-two-swimming-pools-on-site/7596881694.html
The response code is 200
67 out of 9280
https://sfbay.craigslist.org/sby/apa/d/san-jose-professional-co-working-area/7592626560.html
The response code is 200
68 out of 9280
https://sfbay.craigslist.org/sby/apa/d/cupertino-lovely-bedroom-apartment-in/7600320196.html
The response code is 200
69 out of 9280
https://sfbay.craigslist.org/eby/apa/d/fremont-private-decks-resort-pool-sp

https://sfbay.craigslist.org/sfc/apa/d/san-francisco-march-free-top-floor/7600492446.html
The response code is 200
125 out of 9280
https://sfbay.craigslist.org/nby/apa/d/santa-rosa-soaker-tub-ceiling-fans-on/7593831798.html
The response code is 200
126 out of 9280
https://sfbay.craigslist.org/sfc/apa/d/san-francisco-spectacular-coach-house/7600341626.html
The response code is 200
127 out of 9280
https://sfbay.craigslist.org/sfc/apa/d/san-francisco-exceptional-dogpatch/7598002152.html
The response code is 200
128 out of 9280
https://sfbay.craigslist.org/sfc/apa/d/san-francisco-industrial-chic-retreat/7600275989.html
The response code is 200
129 out of 9280
https://sfbay.craigslist.org/pen/apa/d/daly-city-1br-1bath-in-law-for-rent/7595830949.html
The response code is 200
130 out of 9280
https://sfbay.craigslist.org/eby/apa/d/oakland-415-lagunitas-2-condo-quality/7595141044.html
The response code is 200
131 out of 9280
https://sfbay.craigslist.org/eby/apa/d/oakland-on-demand-dry-cleaning-

https://sfbay.craigslist.org/pen/apa/d/san-bruno-coming-soon-bdrm-ba-car/7600180599.html
The response code is 200
187 out of 9280
https://sfbay.craigslist.org/pen/apa/d/mountain-view-beautiful-7th-floor/7593039529.html
The response code is 200
188 out of 9280
https://sfbay.craigslist.org/eby/apa/d/dublin-gracious-947-sq-ft-square-foot/7595804286.html
The response code is 200
189 out of 9280
https://sfbay.craigslist.org/sfc/apa/d/san-francisco-come-and-experience/7600856776.html
The response code is 200
190 out of 9280
https://sfbay.craigslist.org/nby/apa/d/novato-well-loved-home-spacious-custom/7600253714.html
The response code is 200
191 out of 9280
https://sfbay.craigslist.org/pen/apa/d/san-mateo-outdoor-basketball-court/7594773110.html
The response code is 200
192 out of 9280
https://sfbay.craigslist.org/eby/apa/d/fremont-fitness-center-wood-style/7600248006.html
The response code is 200
193 out of 9280
https://sfbay.craigslist.org/eby/apa/d/san-ramon-cable-ready-on-call-and-on/7598

https://sfbay.craigslist.org/sby/apa/d/san-jose-brand-new-community-move-in/7600697735.html
The response code is 200
249 out of 9280
https://sfbay.craigslist.org/pen/apa/d/belmont-gorgeous-complex-updated-unit/7598952915.html
The response code is 200
250 out of 9280
https://sfbay.craigslist.org/eby/apa/d/hayward-bedroom-bath-apartment-only-1995/7599504996.html
The response code is 200
251 out of 9280
https://sfbay.craigslist.org/sby/apa/d/san-jose-air-conditioning-vertical/7598952011.html
The response code is 200
252 out of 9280
https://sfbay.craigslist.org/eby/apa/d/san-ramon-kitchenette-studio-apartment/7600627981.html
The response code is 200
253 out of 9280
https://sfbay.craigslist.org/sby/apa/d/mountain-view-single-story-private/7595012946.html
The response code is 200
254 out of 9280
https://sfbay.craigslist.org/sby/apa/d/san-jose-ultimate-4br-4ba-townhouse/7598606321.html
The response code is 200
255 out of 9280
https://sfbay.craigslist.org/pen/apa/d/daly-city-cozy-br-1bath-in-l

...

https://sfbay.craigslist.org/eby/apa/d/antioch-beutiful-home-for-rent-month-or/7600479567.html
The response code is 200
9053 out of 9280
https://sfbay.craigslist.org/eby/apa/d/fairfield-accepts-credit-card-payments/7600390719.html
The response code is 200
9054 out of 9280
https://sfbay.craigslist.org/sby/apa/d/san-jose-affordable-spacious-with-2-off/7600254522.html
The response code is 200
9055 out of 9280
https://sfbay.craigslist.org/eby/apa/d/berkeley-spacious-1bd-avl-1-b108/7600131717.html
The response code is 200
9056 out of 9280
https://sfbay.craigslist.org/sby/apa/d/santa-clara-1br-1ba-apartment-within/7600454332.html
The response code is 200
9057 out of 9280
https://sfbay.craigslist.org/sfc/apa/d/san-francisco-15ba-video-spacious/7600289850.html
The response code is 200
9058 out of 9280
https://sfbay.craigslist.org/sby/apa/d/hollister-new-construction-complete-1bd/7600851986.html
The response code is 200
9059 out of 9280
https://sfbay.craigslist.org/sby/apa/d/santa-clara-24-hour

https://sfbay.craigslist.org/sby/apa/d/san-jose-visit-your-new-home-updated/7600263270.html
The response code is 200
9115 out of 9280
https://sfbay.craigslist.org/sfc/apa/d/san-francisco-alamo-square-studio-hw/7592323349.html
The response code is 200
9116 out of 9280
https://sfbay.craigslist.org/sfc/apa/d/san-francisco-haight-top-floor-jr-bed/7600416499.html
The response code is 200
9117 out of 9280
https://sfbay.craigslist.org/sfc/apa/d/san-francisco-weeks-free-limited-time/7600397452.html
The response code is 200
9118 out of 9280
https://sfbay.craigslist.org/sfc/apa/d/san-francisco-charming-studio-in/7600590657.html
The response code is 200
9119 out of 9280
https://sfbay.craigslist.org/sby/apa/d/san-jose-gorgeous-1br-loft-san-jose/7597834424.html
The response code is 200
9120 out of 9280
https://sfbay.craigslist.org/sby/apa/d/milpitas-love-where-you-live-weeks-on-us/7590535134.html
The response code is 200
9121 out of 9280
https://sfbay.craigslist.org/eby/apa/d/oakland-east-lake-vict

https://sfbay.craigslist.org/pen/apa/d/mountain-view-visit-this-weekend/7600430236.html
The response code is 200
9177 out of 9280
https://sfbay.craigslist.org/nby/apa/d/santa-rosa-coffee-bar-with-espresso/7600718017.html
The response code is 200
9178 out of 9280
https://sfbay.craigslist.org/eby/apa/d/lafayette-two-bedroom-one-bathroom-home/7600408472.html
The response code is 200
9179 out of 9280
https://sfbay.craigslist.org/eby/apa/d/walnut-creek-spacious-sunny-apartments/7600711225.html
The response code is 200
9180 out of 9280
https://sfbay.craigslist.org/sfc/apa/d/storybook-house-with-parking-4br-25-ba/7600652739.html
The response code is 200
9181 out of 9280
https://sfbay.craigslist.org/eby/apa/d/san-leandro-enjoy-the-gorgeous/7600061680.html
The response code is 200
9182 out of 9280
https://sfbay.craigslist.org/eby/apa/d/san-ramon-public-parks-nearby-vinyl/7600278478.html
The response code is 200
9183 out of 9280
https://sfbay.craigslist.org/eby/apa/d/berkeley-green-certified-fit

https://sfbay.craigslist.org/sby/apa/d/san-jose-bed-bath-3rd-floor-move-in/7600587288.html
The response code is 200
9239 out of 9280
https://sfbay.craigslist.org/sfc/apa/d/san-francisco-sunny-studio-weeks-free/7600485697.html
The response code is 200
9240 out of 9280
https://sfbay.craigslist.org/sby/apa/d/san-jose-1br-1ba-apt-downtown-san-jose/7595519847.html
The response code is 200
9241 out of 9280
https://sfbay.craigslist.org/sby/apa/d/san-jose-1bd-1ba-easy-commute-plus/7600745017.html
The response code is 200
9242 out of 9280
https://sfbay.craigslist.org/sfc/apa/d/san-francisco-spacious-hayes-valley/7600623565.html
The response code is 200
9243 out of 9280
https://sfbay.craigslist.org/pen/apa/d/millbrae-new-pricing-month-free-huge/7600462240.html
The response code is 200
9244 out of 9280
https://sfbay.craigslist.org/sfc/apa/d/san-francisco-convenient-location-deep/7600625297.html
The response code is 200
9245 out of 9280
https://sfbay.craigslist.org/eby/apa/d/oakland-large-updated-

https://sfbay.craigslist.org/eby/apa/d/central-location-quartz-countertops/7600390383.html
The response code is 200
bad connection
https://sfbay.craigslist.org/sby/apa/d/come-look-and-lease-walk-to-san-jose/7600743571.html
The response code is 200
bad connection
https://sfbay.craigslist.org/sfc/apa/d/beautiful-brand-new-fully-remodeled-bed/7600735599.html
The response code is 200
bad connection
https://sfbay.craigslist.org/pen/apa/d/lease-today-non-smoking-community-two/7600693683.html
The response code is 200
bad connection
https://sfbay.craigslist.org/eby/apa/d/visit-our-neighborhood-laundry-in-unit/7600087163.html
The response code is 200
bad connection
https://sfbay.craigslist.org/scz/apa/d/experience-city-views-from-our-sleek/7600726628.html
The response code is 200
bad connection
https://sfbay.craigslist.org/pen/apa/d/call-for-details-bbq-and-picnic-areas/7600779196.html
The response code is 200
bad connection
https://sfbay.craigslist.org/sby/apa/d/stop-by-today-open-floor-plan-d